# **3.d Vectores auxiliares**

**Responsable:**

César Zamora Martínez

**Infraestructura usada:** 
Google Colab, para pruebas


## 0. Importamos librerias necesarias

**Fuente:** 3c_formacion_matrices.ipynb, 3c_formacion_abc.ipynb, 3c_formacion_delta.ipynb

In [1]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   5467      0 --:--:-- --:--:-- --:--:--  5448
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 335.2MB 49kB/s 
+ set +ex
Installation succeeded!


In [0]:
import cupy as cp

def formar_vectores(mu, Sigma):
  '''
  Calcula las cantidades u = \Sigma^{-1}  \mu y v := \Sigma^{-1} \cdot 1 del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    u (cupy array, escalar): vector dado por \cdot Sigma^-1 \cdot mu (dimension n)
    v (cupy array, escalar): vector dado por Sigma^-1 \cdot 1 (dimension n)
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  # Obtiene vectores de interes
  u = cp.linalg.solve(Sigma, mu)
  u = u.transpose()[0] # correcion de expresion de array
  v = cp.linalg.solve(Sigma, ones_vector)

  return u , v

In [0]:
def formar_abc(mu, Sigma):
  '''
  Calcula las cantidades A, B y C del diagrama de flujo del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    A (cupy array, escalar): escalar dado por mu^t \cdot Sigma^-1 \cdot mu
    B (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot 1
    C (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot mu
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  u, v = formar_vectores(mu, Sigma)

  # Obtiene escalares de interes
  A = mu.transpose()@u
  B = ones_vector.transpose()@v
  C = ones_vector.transpose()@u

  return A, B, C

In [0]:
def delta(A,B,C):
  '''
  Calcula las cantidad Delta = AB-C^2 del diagrama de flujo del problema de Markowitz

  Args:
    A (cupy array, escalar): escalar dado por mu^t \cdot Sigma^-1 \cdot mu
    B (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot 1
    C (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot mu

  Return:
    Delta (cupy array, escalar): escalar dado \mu^t \cdot \Sigma^{-1} \cdot \mu
  '''
  Delta = A*B-C**2

  return Delta

## 1. Implementación

**Consideraciones:**. Esta etapa supone que se conocen $\bar{r}$, $\mu$ y $\Sigma$ asociados a los activos, ello con el objeto de es obtener valores escalares que serán relevantes para obtener los pesos del portafolio para el inversionista. Hasta este punto se asume que ya conocemos todos los términos presentes en las expresiones:

$$A = \mu^t \cdot \Sigma^{-1} \cdot \mu $$

$$B = 1^t \cdot \Sigma^{-1} \cdot 1 $$

$$C = 1^t \cdot \Sigma^{-1} \cdot \mu = \mu^t \cdot \Sigma^{-1} \cdot 1 $$

Para con ello poder estimar los multiplicadores de Lagrange asociados al problema:

$$ w_0 = \frac{1}{\Delta} ( \hat{r} \cdot B - C ) $$

$$ w_1 = \frac{1}{\Delta} (A - C \cdot \hat{r}) $$

Con los que se forma la solución del sistema dada por

$$w = w_0 \cdot (\Sigma^{-1} \mu) + w_1 \cdot (\Sigma^{-1} 1) $$

En seguida se presenta el código correspondiente:

In [0]:
def formar_omegas(r, mu, Sigma):
  '''
  Calcula las cantidades w_o y w_ del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    w_0 (cupy array, matriz): matriz dada por 
          w_0 = \frac{1}{\Delta} (B \Sigma^{-1} \hat{\mu}- C\Sigma^{-1} 1) 
    w_1 (cupy array, vector): vector dado por 
         w_1 = \frac{1}{\Delta} (C \Sigma^{-1} \hat{\mu}- A\Sigma^{-1} 1)
  '''
  # Obtenemos u = Sigma^{-1} \hat{\mu}, v = \Sigma^{-1} 1
  u, v = formar_vectores(mu, Sigma)
  # Escalares relevantes
  A, B, C = formar_abc(mu, Sigma)
  Delta = delta(A,B,C)
  # Formamos w_0 y w_1
  w_0 = (1/Delta)*(r*B-C)
  w_1 = (1/Delta)*(A-C*r)

  return w_0, w_1

## 1.1 Valores de prueba

In [6]:
n= 10

# r y mu
r= 10
mu=cp.random.rand(n, 1)

# Sigma
S=cp.random.rand(n, n)
Sigma=S@S

# multiplicadores de lagrande
formar_omegas(r,mu,Sigma)

(array([-0.47434621]), array([3.37222863]))